# Import

In [1]:
import os, gc, re, csv
from collections import defaultdict

import pandas as pd
from tqdm.auto import tqdm
import numpy as np

from collections import Counter
import math

from Bio import SeqIO

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.multiclass import OneVsRestClassifier

c:\Users\TuanLe\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Parsing data

## Parse GO graph

In [2]:
# Copied from https://www.kaggle.com/code/seddiktrk/cafa-6-blend-goa-negative-propagation/notebook

def parse_obo(go_obo_path):
    parents = defaultdict(set)
    children = defaultdict(set)

    if not os.path.exists(go_obo_path):
        return parents, children

    with open(go_obo_path, "r") as f:
        cur_id = None
        for line in f:
            line = line.strip()
            if line == "[Term]":
                cur_id = None
            elif line.startswith("id: "):
                cur_id = line.split("id: ")[1].strip()
            elif line.startswith("is_a: "):
                pid = line.split()[1].strip()
                if cur_id:
                    parents[cur_id].add(pid)
                    children[pid].add(cur_id)
            elif line.startswith("relationship: part_of "):
                parts = line.split()
                if len(parts) >= 3:
                    pid = parts[2].strip()
                    if cur_id:
                        parents[cur_id].add(pid)
                        children[pid].add(cur_id)
    print(f"[io] Parsed OBO: {len(parents)} nodes with parents")
    return parents, children


def get_ancestors(go_id, parents):
    ans = set()
    stack = [go_id]
    while stack:
        cur = stack.pop()
        for p in parents.get(cur, []):
            if p not in ans:
                ans.add(p)
                stack.append(p)
    return ans


def get_descendants(go_id, children):
    desc = set()
    stack = [go_id]
    while stack:
        cur = stack.pop()
        for child in children.get(cur, []):
            if child not in desc:
                desc.add(child)
                stack.append(child)
    return desc

## Parse train data

In [3]:
terms_df = pd.read_csv(os.path.join("./Train/train_terms.tsv"), sep="\t", usecols=["EntryID", "term"])
train_annotations = terms_df.groupby("EntryID")["term"].apply(list).to_dict()

train_sq = []
train_answer = []

terms_to_answer = terms_df.groupby('EntryID')['term'].apply(list).to_dict()

cnt = 0


for record in SeqIO.parse(os.path.join("./Train/train_sequences.fasta"), "fasta"):
    try:
        if "|" in record.id:
            clean_id = record.id.split("|")[1]
        else:
            clean_id = record.id
        a = record.description.split("OX=")
        b = a[1].split(" ")[0]
        if clean_id:
            train_sq.append({
                "id": clean_id,
                "tax": b,
                "seq": str(record.seq),
                "answer": terms_to_answer[clean_id]
            })
        else:
            print("123")
    except IndexError:
        continue

test_sq = []

for record in SeqIO.parse(os.path.join("./Test/testsuperset.fasta"), "fasta"):
    tax = record.description.split(" ")[1]
    # if (tax not in test_gr):
    #     test_gr[tax] = []
    test_sq.append({
        "id": record.id,
        "tax": tax,
        "seq": str(record.seq)
    })

test_df = pd.DataFrame(test_sq)
train_df = pd.DataFrame(train_sq)

In [4]:
print(len(train_df), len(test_df))
print(terms_df[:10])
print(train_df[:10])
print(test_df[:10])

82404 224309
  EntryID        term
0  Q5W0B1  GO:0000785
1  Q5W0B1  GO:0004842
2  Q5W0B1  GO:0051865
3  Q5W0B1  GO:0006275
4  Q5W0B1  GO:0006513
5  Q5W0B1  GO:0003682
6  Q5W0B1  GO:0005515
7  Q3EC77  GO:0000138
8  Q3EC77  GO:0005794
9  Q8IZR5  GO:0005515
           id   tax                                                seq  \
0  A0A0C5B5G6  9606                                   MRWQEMGYIFYPRKLR   
1      A0JNW5  9606  MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...   
2      A0JP26  9606  MVAEVCSMPAASAVKKPFDLRSKMGKWCHHRFPCCRGSGKSNMGTS...   
3      A0PK11  9606  MPGWFKKAWYGLASLLSFSSFILIIVALVVPHWLSGKILCQTGVDL...   
4      A1A4S6  9606  MGLQPLEFSDCYLDSPWFRERIRAHEAELERTNKFIKELIKDGKNL...   
5      A1A519  9606  MKRRQKRKHLENEESQETAEKGGGMSKSQEDALQPGSTRVAKGWSQ...   
6      A1L190  9606  MDDADPEERNYDNMLKMLSDLNKDLEKLLEEMEKISVQATWMAYDM...   
7      A1L3X0  9606  MAFSDLTSRTVHLYDNWIKDADPRVEDWLLMSSPLPQTILLGFYVY...   
8      A1X283  9606  MPPRRSIVEVKVLDVQKRRVPNKHYVYIIRVTWSSGSTEAIYRRYS...   
9    

# Getting features

In [5]:
def get_bigrams(seq):
    return [seq[i:i+2] for i in range(len(seq)-1)]

def build_idf(train_seqs, vocab):
    df = Counter()
    total_docs = len(train_seqs)

    for seq in train_seqs:
        bigs = set(get_bigrams(seq))
        for b in bigs:
            if b in vocab:
                df[b] += 1

    idf = {}
    for b in vocab:
        idf[b] = math.log((1 + total_docs) / (1 + df[b])) + 1

    return idf

def seq_to_tfidf_vector(seq, vocab, idf, norm=True):
    bigs = get_bigrams(seq)
    count = Counter(bigs)

    vec = np.zeros(len(vocab), dtype=float)

    for i, b in enumerate(vocab):
        tf = count[b]
        vec[i] = tf * idf[b]

    if norm:
        s = np.linalg.norm(vec)
        if s > 0:
            vec = vec / s

    return vec

def build_feature_matrix(seqs, vocab, idf):
    return np.array([seq_to_tfidf_vector(seq, vocab, idf) for seq in seqs])



In [6]:
features = ['LL', 'SS', 'SL', 'LS',
            'SSS', 'LLL', 'AAA', 'EEE', 'LLS', 'SLL', 'SSL', 'PPP', 'GGG', 'LLA', 'LSS', 'ALL', 'LSL', 'SLS', 'QQQ', 'LAL', 'LAA', 'NNN', 'LLE', 'LLG', 'AAL', 'ELL', 'EEL', 'ALA', 'LEE', 'LGL', 'VLL', 'GLL', 'LLK', 'SGS', 'ASS', 'GSS', 'LLV', 'SSG', 'LAS', 'RLL', 'LVL', 'LLR', 'SSA', 'SAS', 'PSS', 'KLL', 'EAL', 'TSS', 'SLA', 'LEK', 'LLD']


print(len(features))


idf = build_idf(train_df["seq"], features)
X_train = build_feature_matrix(train_df["seq"], features, idf)

51


# Get label

In [7]:
mlb = MultiLabelBinarizer(sparse_output=True)
y_train = mlb.fit_transform(train_df["answer"])
print(y_train.shape)

(82404, 26125)


# Random Forest

In [8]:
clf = OneVsRestClassifier(LogisticRegression(max_iter=500, random_state=42, n_jobs=-1), n_jobs=-1, verbose=1)
clf.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   26.1s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   42.3s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 6034 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 7184 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 8434 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 9784 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 11234 tasks      |

,estimator,LogisticRegre...ndom_state=42)
,n_jobs,-1
,verbose,1
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42


# Get output from model

In [9]:
X_test = build_feature_matrix(test_df["seq"], features, idf)


In [10]:

arr = test_df['id']

In [11]:
batch_size = 32768   # hoặc 10, 32 tùy RAM
threshold = 0.02  # THRESHOLD BẠN MUỐN

all_probs = []

for i in range(0, X_test.shape[0], batch_size):
    print(f"Predict for [{i}, {i+batch_size}]")
    X_batch = X_test[i:i+batch_size]

    # predict proba
    prob_batch = clf.predict_proba(X_batch)  # shape (batch, n_classes)

    # lọc theo threshold
    for j, prob_vec in enumerate(prob_batch):
        cur_id = arr[i+j]
        filtered = []
        for class_idx, p in enumerate(prob_vec):
            if p >= threshold:
                label = mlb.classes_[class_idx]
                filtered.append((label, float(p)))   # (GO_label, prob)


        all_probs.append({"id": cur_id, "preds": filtered})

Predict for [0, 32768]
Predict for [32768, 65536]
Predict for [65536, 98304]
Predict for [98304, 131072]
Predict for [131072, 163840]
Predict for [163840, 196608]
Predict for [196608, 229376]


In [12]:
all_probs[:20]

[{'id': 'A0A0C5B5G6',
  'preds': [('GO:0002181', 0.030164343722657555),
   ('GO:0003723', 0.034401188361547014),
   ('GO:0003729', 0.028194566907640665),
   ('GO:0003735', 0.09885721091605557),
   ('GO:0005515', 0.2534973473522968),
   ('GO:0005576', 0.20073422115943418),
   ('GO:0005615', 0.04116135526348017),
   ('GO:0005634', 0.12133336648204028),
   ('GO:0005654', 0.035750068816940275),
   ('GO:0005730', 0.03100480821908945),
   ('GO:0005737', 0.08562874033449303),
   ('GO:0005739', 0.0777681479309486),
   ('GO:0005743', 0.03078987555668573),
   ('GO:0005829', 0.17957136118666644),
   ('GO:0005886', 0.030225445536400785),
   ('GO:0006412', 0.02000049373200165),
   ('GO:0006952', 0.023290941436830023),
   ('GO:0009536', 0.035928757565406276),
   ('GO:0022625', 0.07017917076224345),
   ('GO:0022626', 0.03542076951711042),
   ('GO:0022627', 0.0464273850886631),
   ('GO:0042802', 0.02840993921358546),
   ('GO:0050829', 0.04858603127561704),
   ('GO:0050830', 0.046997915675289825),
   (

In [13]:
# all_probs[:20]

output_file = "submission.tsv"

with open(output_file, "w") as f:
    for mp in all_probs:
        # mp['id'], mp['pred']
        for pred, prob in mp['preds']:
            f.write(mp['id']+"\t"+pred+"\t"+str(round(prob, 3))+"\n")

# with open(output_file, "w", newline='', encoding="utf-8") as f:
#     writer = csv.writer(f)
#     writer.writerow(["id", "label", "probability"])  # header

#     for sample_idx, sample_id in enumerate(ids):
#         for label_idx, label in enumerate(labels):
#             prob = probs[sample_idx][label_idx]
#             if prob >= threshold:
#                 writer.writerow([sample_id, label, float(prob)])